In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
path='/content/drive/My Drive/ML_COURSE/HW8/'

In [ ]:
train = pd.read_csv(path+"train.tsv", sep ='\t')

In [ ]:
train.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,framebased,frameTagRatio,hasDomainLink,html_ratio,image_ratio,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,0.443783,0.0,0,0.090774,0,0.245831,0.003883,1,1,24,0,5424,170,8,0.152941,0.079130,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,0.468649,0.0,0,0.098707,0,0.203490,0.088652,1,1,40,0,4973,187,9,0.181818,0.125448,1
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,0.525448,0.0,0,0.072448,0,0.226402,0.120536,1,1,55,0,2240,258,11,0.166667,0.057613,1
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,0.480725,0.0,0,0.095861,0,0.265656,0.035343,1,0,24,0,2737,120,5,0.041667,0.100858,1
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,0.446143,0.0,0,0.024908,0,0.228887,0.050473,1,1,14,0,12032,162,10,0.098765,0.082569,0


In [ ]:
train.describe()

,urlid,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,framebased,frameTagRatio,hasDomainLink,html_ratio,image_ratio,lengthyLinkDomain,linkwordscore,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
count,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.0,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000
mean,5305.704665,2.761823,0.468230,0.214080,0.092062,0.049262,2.255103,-0.103750,0.0,0.056423,0.021231,0.233778,0.275709,0.660311,30.077079,5716.598242,178.754564,4.960649,0.172864,0.101221,0.513320
std,3048.384114,8.619793,0.203133,0.146743,0.095978,0.072629,5.704313,0.306545,0.0,0.041446,0.144162,0.052487,1.919320,0.473636,20.393101,8875.432430,179.466198,3.233111,0.183286,0.079231,0.499856
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0,0.000000,0.000000,0.045564,-1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2688.500000,1.602062,0.340370,0.105263,0.022222,0.000000,0.442616,0.000000,0.0,0.028502,0.000000,0.201061,0.025900,0.000000,14.000000,1579.000000,82.000000,3.000000,0.040984,0.068739,0.000000
50%,5304.000000,2.088235,0.481481,0.202454,0.068627,0.022222,0.483680,0.000000,0.0,0.045775,0.000000,0.230564,0.083051,1.000000,25.000000,3500.000000,139.000000,5.000000,0.113402,0.089312,1.000000
75%,7946.500000,2.627451,0.616604,0.300000,0.133333,0.065065,0.578227,0.000000,0.0,0.073459,0.000000,0.260770,0.236700,1.000000,43.000000,6377.000000,222.000000,7.000000,0.241299,0.112376,1.000000
max,10566.000000,363.000000,1.000000,1.000000,0.980392,0.980392,21.000000,0.250000,0.0,0.444444,1.000000,0.716883,113.333333,1.000000,100.000000,207952.000000,4997.000000,22.000000,1.000000,1.000000,1.000000


<h1>Вариант с обработкой только одного признака (boilerplate)<br> Без предобработки текста

In [ ]:
import numpy as np
from sklearn import metrics,preprocessing,model_selection
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.linear_model as lm
import pandas as p

path='/content/drive/My Drive/ML_COURSE/HW8/'



print("loading data..")
traindata = list(np.array(p.read_table(path+'train.tsv'))[:,2])
testdata = list(np.array(p.read_table(path+'test.tsv'))[:,2])
y = np.array(p.read_table(path+'train.tsv'))[:,-1]

y=y.astype('int')

tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode',  
      analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1)

rd = lm.LogisticRegression(penalty='l2', dual=True, tol=0.0001, 
                           C=1, fit_intercept=True, intercept_scaling=1.0, 
                           class_weight=None, random_state=None)

X_all = traindata + testdata
lentrain = len(traindata)

print("fitting pipeline")
tfv.fit(X_all)
print("transforming data")
X_all = tfv.transform(X_all)

X = X_all[:lentrain]
X_test = X_all[lentrain:]

           
print("20 Fold CV Score: ", np.mean(cross_val_score(rd, X, y, cv=20, scoring='roc_auc')))

print("training on full data")
rd.fit(X,y)
pred = rd.predict_proba(X_test)[:,1]
testfile = p.read_csv(path+'test.tsv', sep="\t", na_values=['?'], index_col=1)
pred_df = p.DataFrame(pred, index=testfile.index, columns=['label'])
pred_df.to_csv('benchmark.csv')
print("submission file created..")



loading data..


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  from ipykernel import kernelapp as app


fitting pipeline
transforming data


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

20 Fold CV Score:  0.8771223762342266
training on full data


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


submission file created..


In [ ]:
traindata

<h1>20 Fold CV Score:  0.8771223762342266

<h1>Делаем небольшую предобработку с 

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer 
import re


def tokenize_stem_lem(text):
    lemma = nltk.wordnet.WordNetLemmatizer()
    stemmer = nltk.stem.PorterStemmer()
    """
    First tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    """
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            if 'title' == token:
                token = ''
            if 'url' == token:
                token = ''
            if 'titl' == token:
                token = ''
            filtered_tokens.append(token)
    stems = [lemma.lemmatize(stemmer.stem(t)) for t in filtered_tokens if len(t) > 0]
    return stems 

In [ ]:
#from preprocessing import preprocess_pipeline

#Change and add following:

print("loading data..")
traindata = list(np.array(p.read_table(path+'train.tsv'))[:,2])
testdata = list(np.array(p.read_table(path+'test.tsv'))[:,2])
y = np.array(p.read_table(path+'train.tsv'))[:,-1]

y=y.astype('int')


print("pre-processing data")


traindataraw = []
testdataraw = []

for observation in traindata:
  arr = tokenize_stem_lem(observation)
  s = ' '.join(arr)
  traindataraw.append(s.lower())
for observation in testdata:
  arr = tokenize_stem_lem(observation)
  s = ' '.join(arr)
  testdataraw.append(s.lower())

tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode',  
      analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1)

rd = lm.LogisticRegression(penalty='l2', dual=True, tol=0.0001, 
                           C=1, fit_intercept=True, intercept_scaling=1.0, 
                           class_weight=None, random_state=None)

X_all = traindataraw + testdataraw
lentrain = len(traindata)

print("fitting pipeline")
tfv.fit(X_all)
print("transforming data")
X_all = tfv.transform(X_all)

X = X_all[:lentrain]
X_test = X_all[lentrain:]

           
print("20 Fold CV Score: ", np.mean(cross_val_score(rd, X, y, cv=20, scoring='roc_auc')))

print("training on full data")
rd.fit(X,y)
pred = rd.predict_proba(X_test)[:,1]
testfile = p.read_csv(path+'test.tsv', sep="\t", na_values=['?'], index_col=1)
pred_df = p.DataFrame(pred, index=testfile.index, columns=['label'])
pred_df.to_csv('benchmark.csv')
print("submission file created..")




loading data..


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """


pre-processing data
fitting pipeline
transforming data


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

20 Fold CV Score:  0.8768655886474583
training on full data


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


submission file created..


<h1>20 Fold CV Score:  0.8768655886474583 <br> Тоесть предобработка чуть хуже дает р-т

<h1>Добавим bag ансамбли из LogisticRegression

In [ ]:
print("loading data..")
traindata = list(np.array(p.read_table(path+'train.tsv'))[:,2])
testdata = list(np.array(p.read_table(path+'test.tsv'))[:,2])
y = np.array(p.read_table(path+'train.tsv'))[:,-1]

y=y.astype('int')


tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode',  
      analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1)

###rd = lm.LogisticRegression(penalty='l2', dual=True, tol=0.0001,                            C=1, fit_intercept=True, intercept_scaling=1.0,                            class_weight=None, random_state=None)

clf  = lm.LogisticRegression(penalty='l2', dual=True, tol=0.0001,  max_iter=5000, C=1, fit_intercept=True, intercept_scaling=1.0,                            class_weight=None, random_state=None)

seed = 7
kfold = model_selection.KFold(n_splits=20, random_state=seed)
num_trees = 20
rd = BaggingClassifier(clf, 20, 0.75,0.75,False,False)


X_all = traindata + testdata
lentrain = len(traindata)

print("fitting pipeline")
tfv.fit(X_all)
print("transforming data")
X_all = tfv.transform(X_all)

X = X_all[:lentrain]
X_test = X_all[lentrain:]

           
#print("20 Fold CV Score: ", np.mean(cross_val_score(rd, X, y, cv=20, scoring='roc_auc')))
results = model_selection.cross_val_score(rd, X, y, cv=kfold)
print(results.mean())


print("training on full data")
rd.fit(X,y)
pred = rd.predict_proba(X_test)[:,1]
testfile = p.read_csv(path+'test.tsv', sep="\t", na_values=['?'], index_col=1)
pred_df = p.DataFrame(pred, index=testfile.index, columns=['label'])
pred_df.to_csv('benchmark.csv')
print("submission file created..")





loading data..


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.


fitting pipeline
transforming data


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWar

<h1> Получили 0.48667318875032006 что намного хуже

<h1>DecisionTreeClassifier()
  
  https://machinelearningmastery.com/ensemble-machine-learning-algorithms-python-scikit-learn/

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

print("loading data..")
traindata = list(np.array(p.read_table(path+'train.tsv'))[:,2])
testdata = list(np.array(p.read_table(path+'test.tsv'))[:,2])
y = np.array(p.read_table(path+'train.tsv'))[:,-1]

y=y.astype('int')


tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode',  
      analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1)

###rd = lm.LogisticRegression(penalty='l2', dual=True, tol=0.0001,                            C=1, fit_intercept=True, intercept_scaling=1.0,                            class_weight=None, random_state=None)

clf  = DecisionTreeClassifier()

seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
num_trees = 20
rd = BaggingClassifier(base_estimator=clf, n_estimators=num_trees, random_state=seed)

X_all = traindata + testdata
lentrain = len(traindata)

print("fitting pipeline")
tfv.fit(X_all)
print("transforming data")
X_all = tfv.transform(X_all)

X = X_all[:lentrain]
X_test = X_all[lentrain:]

           
#print("20 Fold CV Score: ", np.mean(cross_val_score(rd, X, y, cv=20, scoring='roc_auc')))
results = model_selection.cross_val_score(rd, X, y, cv=kfold)
print('RESULT==',results.mean())


print("training on full data")
rd.fit(X,y)
pred = rd.predict_proba(X_test)[:,1]
testfile = p.read_csv(path+'test.tsv', sep="\t", na_values=['?'], index_col=1)
pred_df = p.DataFrame(pred, index=testfile.index, columns=['label'])
pred_df.to_csv('benchmark.csv')
print("submission file created..")



loading data..


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


fitting pipeline
transforming data
RESULT== 0.7863478769703398
training on full data
submission file created..


<h1> DecisionTreeClassifier BAG:<br>0.7863478769703398

<h1>Gradient Boosting classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

print("loading data..")
traindata = list(np.array(p.read_table(path+'train.tsv'))[:,2])
testdata = list(np.array(p.read_table(path+'test.tsv'))[:,2])
y = np.array(p.read_table(path+'train.tsv'))[:,-1]

y=y.astype('int')


tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode',  
      analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1)


seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
num_trees = 100

rd  = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed)



X_all = traindata + testdata
lentrain = len(traindata)

print("fitting pipeline")
tfv.fit(X_all)
print("transforming data")
X_all = tfv.transform(X_all)

X = X_all[:lentrain]
X_test = X_all[lentrain:]

           
#print("20 Fold CV Score: ", np.mean(cross_val_score(rd, X, y, cv=20, scoring='roc_auc')))
results = model_selection.cross_val_score(rd, X, y, cv=kfold)
print(results.mean())


print("training on full data")
rd.fit(X,y)
pred = rd.predict_proba(X_test)[:,1]
testfile = p.read_csv(path+'test.tsv', sep="\t", na_values=['?'], index_col=1)
pred_df = p.DataFrame(pred, index=testfile.index, columns=['label'])
pred_df.to_csv('benchmark.csv')
print("submission file created..")


loading data..


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


fitting pipeline
transforming data
0.7921594192297846
training on full data
submission file created..


<h1> GradientBoosting 0.7921594192297846